In [1]:
from MFEA_lib.model import MFEA_base, SA_MFEA, SMP_MFEA, MFEA1
from MFEA_lib.model.utils import *
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *

In [2]:
import sys
MAX_INT = sys.maxsize
class TULKH_FUNC(AbstractTask):    
    def __init__(self, path_data):
        self.datas = {}
        self.N: int
        self.e: np.ndarray
        self.l: np.ndarray
        self.d: np.ndarray
        self.path_data = path_data
        self.read_data()
            
    def read_data(self):
        with open(self.path_data, 'r') as file: 
            n = int(file.readline())
            early = []
            late = []
            delay = []
            cost = []
            time = []
            for i in range(n+1):
                e, l, d = file.readline().strip().split(' ')
                early.append(int(e))
                late.append(int(l))
                delay.append(int(d))
            for i in range(n+1):
                costi = [int(a) for a in file.readline().strip().split(' ')]
                cost.append(costi)
            for i in range(n+1):
                timei = [int(a) for a in file.readline().strip().split(' ')]
                time.append(timei)
        data = {}
        data['n'] = n
        data['early'] = np.array(early)
        data['late'] = np.array(late)
        data['delay'] = np.array(delay)
        data['cost'] = np.array(cost)
        data['time'] = np.array(time)
        self.dim = n
        self.data = data


    @staticmethod
    def func(gene,
             data,
             ):
        cost = 0 
        idx = np.argsort(-gene)
        curr = 0
        curr_time = 0
        visited_vertex = [False for i in range(data['n'] + 1)]
        num_visited = 0
        while (num_visited != data['n']):
            stop = True
            for t in idx:
                try:
                    if visited_vertex[t]:
                        continue
                except:
                    print(t, len(visited_vertex))
                arrive_time = curr_time + data['time'][curr][t]
                if arrive_time > data['late'][t]:
                    continue
                
                cost += data['cost'][curr][t]
                curr = t
                cur_time = max(arrive_time, data['early'][t])
                cur_time += data['delay'][t]
                visited_vertex[t] = True
                num_visited += 1
                stop = False
                
            if stop:
                return MAX_INT
        
        return cost
        
    def __call__(self, gene: np.ndarray):
        # decode
        gene = gene[ : self.dim]
        # eval
        return __class__.func(gene, self.data)

In [3]:
from tqdm import tqdm
class Ind_TULKH(Individual):
    def __init__(self, genes, dim=None) -> None:
        super().__init__(genes, dim)
        if genes is None:
            self.genes: np.ndarray = np.random.permutation(dim)  + 1
            
class TULKH_benchmark:
    def get_tasks():
        print('\rReading data...')
        tasks = []
        file_list = ['DATA/' + file_name for file_name in ['50points_5days_DoubleDiscrepancy.txt', '10points_1day.txt', '25points_3days.txt']]
        for file_name in tqdm(file_list):
            tasks.append(TULKH_FUNC(file_name))
                         
        return tasks, Ind_TULKH

In [4]:
tasks, IndClass = TULKH_benchmark.get_tasks()


Reading data...


100%|██████████| 3/3 [00:00<00:00, 451.00it/s]


In [5]:
class TULKH_Crossover(AbstractCrossover):
    def __call__(self, pa: Individual, pb: Individual, skf_oa=None, skf_ob=None, *args, **kwargs) -> Tuple[Individual, Individual]:
        genes_oa, genes_ob = np.empty_like(pa), np.empty_like(pb)

        #PMX
        t1, t2 = np.random.randint(0, self.dim_uss + 1, 2)
        if t1 > t2:
            t1, t2 = t2, t1
        genes_oa, genes_ob = pmx_func(pa.genes, pb.genes, t1, t2, self.dim_uss)

        oa = self.IndClass(genes_oa)
        ob = self.IndClass(genes_ob)

        oa.skill_factor = skf_oa
        ob.skill_factor = skf_ob
        return oa, ob

class TULKH_Mutation(AbstractMutation):
    def getInforTasks(self, IndClass: Type[Individual], tasks: list[AbstractTask], seed=None):
        super().getInforTasks(IndClass, tasks, seed)
        
    def __call__(self, ind: Individual, return_newInd: bool, *arg, **kwargs) -> Individual:
        return ind

In [6]:
baseModel = MFEA_base.model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover= newSBX(nc = 2, gamma= 0.4, alpha= 6),
    crossover= TULKH_Crossover(),
    mutation= TULKH_Mutation(),
    selection= ElitismSelection()
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.3, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Time: 00m 0.24s    0 % [>                   ]  Cost: 9.22E+18  9.22E+18  9.22E+18  ,  

: 

: 